In [1]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates


from QUANTAXIS.QAUtil import DATABASE
from QUANTAXIS.QAUtil import  trade_date_sse
from QUANTAXIS.QAUtil.QADate_trade import (
    QA_util_get_pre_trade_date,
    QA_util_get_next_trade_date,
    QA_util_if_tradetime
)

import ind.alpha191 as a191
from scipy import stats
import statsmodels.api as sm

# import cpuinfo
# if 'ntel' in cpuinfo.get_cpu_info()['brand_raw']:
# from sklearnex import patch_sklearn, unpatch_sklearn
# unpatch_sklearn() ##注意，少量数据的线性回归没有优势。慎用，存在内存泄露

from sklearn import linear_model


%load_ext autoreload
%autoreload 2
%aimport tools.Cacher


In [2]:
def pretreate_data(data):
    returns = smpl.get_current_return(data,'close')
    returns.name = 'returns'
    ret_forward = smpl.get_forward_return(data,'close')
    ret_forward.name = 'ret_forward'
    # {'Open', 'cap', 'close', 'high', 'ind', 'low', 'returns', 'volume', 'vwap'}
    data = pd.concat([data, returns, ret_forward], axis=1)
    data = data.assign(vwap=data.amount/(data.volume*100))
    data = smpl.add_marketvalue_industry(data)
    data.rename(columns = {"open":"Open",'totalCapital':'cap','industry':'ind'}, inplace=True)
    
    data.rename(columns = {"open":"Open"}, inplace=True)
    data = smpl.add_report_inds(data,'netAssetsPerShare')
    data['pb']=data['close']/data['netAssetsPerShare']
    return data
    # return pd.concat([data, close_ind, vwap_ind, low_ind, high_ind, volume_ind, adv20_ind, adv40_ind, adv81_ind, co_mixed_ind, oh_mixed_ind, lv_mixed_ind], axis=1)




In [3]:
def assemble_stocks_by_codes(codes):
    files = list(map(lambda x:x+'_train_qfq',l))
    return pd.concat(list(map(lambda file:load_cache(file,cache_type=CACHE_TYPE.STOCK),files))).sort_index(level=0)

l = smpl.get_codes_from_blockname('沪深300', sse='all')
df_all =  assemble_stocks_by_codes(l)

In [4]:
df_treated = pretreate_data(df_all)

In [20]:
# ids = [11,24,38,41,42,47,57,69,80,82,83,88,93,97]
ids = [11,24]

# ids = np.arange(1,101)
# np.random.shuffle(ids)

# def generate_alpha_factors(fun_ids, df=None):
#     import ind.alpha191 as a191
#     from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)
#     from base.JuUnits import excute_for_multidates

# for i in fun_ids:
for i in ids:
    fun_name = 'alpha'+ '0'*(3- int(np.log10(i)+1)) +str(i)
    # indx = excute_for_multidates(df, lambda x: getattr(a191, fun_name)(x) ,level=1)
    indx = excute_for_multidates(df_treated, lambda x: getattr(a191, fun_name)(x) ,level=1)
    indx.name = fun_name
    # save_cache('{}_train'.format(fun_name), indx, cache_type=CACHE_TYPE.FACTOR)
    print(indx)
# worker=6
# task = task_chunk_split(ids, worker)
# results = parallal_task(worker, generate_alpha_factors, task, df=df_treated)


code
000001   -4.506804e+07
000002    1.596329e+07
000063   -3.630724e+07
000066    3.796777e+06
000069    2.500154e+07
              ...     
603882   -1.483158e+05
603899   -2.834211e+06
603986    1.431323e+05
603993   -1.358729e+08
605499   -4.952623e+05
Name: alpha011, Length: 248, dtype: float64
code
000001   -0.406365
000002   -0.211400
000063    0.276363
000066    0.133742
000069    0.348468
            ...   
603882    1.635349
603899    1.838936
603986    0.895157
603993   -0.114700
605499   -1.127755
Name: alpha024, Length: 248, dtype: float64


In [5]:
# excute_for_multidates(df_treated.sort_index(), lambda x: a191.alpha011(x) ,level=1)
def STD(data, windows):
    return data.rolling(window=windows, min_periods=windows).std()
def MEAN(data, windows):
    return data.rolling(window=windows, min_periods=windows).mean()
def DELTA(data, windows):
    return data.diff(4)
def SEQUENCE(n):
    return pd.Series(np.arange(1,n+1))

def SMA(data,windows,alpha):
    return data.ewm(adjust=False, alpha=float(alpha)/windows, min_periods=windows, ignore_na=False).mean()

def REGBETA(xs, y, n):
    assert len(y)>=n,  'len(y)!>=n !!!'
    regress = linear_model.LinearRegression(fit_intercept=False)
    def reg(X,Y):
        try:
            if len(Y)>len(X):
                res = regress.fit(X.values.reshape(-1, 1), Y[X.index].values.reshape(-1, 1)).coef_[0]
            else:
                res = regress.fit(X.values.reshape(-1, 1), Y.values.reshape(-1, 1)).coef_[0]
        except Exception as e:
            print(e)
            return np.nan
        return res
    return xs.rolling(window=n, min_periods=n).apply(lambda x:reg(x,y))


def COVIANCE(A,B,d):
    se = pd.Series(np.arange(len(A.index)),index=A.index)
    se = se.rolling(5).apply(lambda x: A.iloc[x].cov(B.iloc[x]))
    return se

def CORR(A,B,d):
    se = pd.Series(np.arange(len(A.index)),index=A.index)
    se = se.rolling(5).apply(lambda x: A.iloc[x].corr(B.iloc[x]))
    return se


def alpha021(data, max_window=10):
    # REGBETA(MEAN(CLOSE,6),SEQUENCE(6))
    a = MEAN(data['close'], 6)
    a = REGBETA(a,SEQUENCE(6),6)
    return a

def alpha019(data, dependencies=['close'], max_window=6):
    # (CLOSE<DELAY(CLOSE,5)?(CLOSE/DELAY(CLOSE,5)-1):(CLOSE=DELAY(CLOSE,5)?0:(1-DELAY(CLOSE,5)/CLOSE)))
    # 类似于近五日涨幅
    condition1 = data['close'] <= data['close'].shift(5)
    alpha = pd.Series(np.nan, index= data['close'].index)
    alpha[condition1] = data['close'].pct_change(periods=5)[condition1]
    alpha[~condition1] = -data['close'].pct_change(periods=5)[~condition1]
    return alpha

print(alpha021(df_treated.loc[(slice(None),['000001']),:]).to_list())


[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 0.4158441605990718, 0.41584047831574167, 0.4157680374805261, 0.41560176866261883, 0.41534129871146425, 0.41492641850680745, 0.4143504668742546, 0.41364514980913386, 0.41286834107948844, 0.41199304969411077, 0.41120300514319463, 0.41047569907212583, 0.4099710385676417, 0.40949459994695636, 0.409067639543281, 0.4086438620048905, 0.4080743003390178, 0.40739314456277714, 0.4064176970824788, 0.4054566719405226, 0.40436119573560453, 0.4032873483900918, 0.4022946011382463, 0.40123474697108447, 0.40050997708810954, 0.40005642812555503, 0.39999536514511835, 0.4001638083665217, 0.40055390030309546, 0.401175274552361, 0.40177831389698593, 0.4023725603391705, 0.402798778306776, 0.40306656618083353, 0.40328375383330534, 0.4035898334630045, 0.40376648491932754, 0.403698470540418, 0.403589529436268, 0.40366229920184266, 0.40390435713502376, 0.4045042434887931, 0.40576070785329454, 0.40737467965317714, 0.4091798733141506, 0.41104768525234575, 0.412733

In [21]:
# a = df_treated.loc[(slice(None),['000001']),:]
alpha021(a)

date        code  
2017-01-04  000001         NaN
2017-01-05  000001         NaN
2017-01-06  000001         NaN
2017-01-09  000001         NaN
2017-01-10  000001         NaN
                        ...   
2021-12-24  000001    0.201586
2021-12-27  000001    0.202608
2021-12-28  000001    0.203365
2021-12-29  000001    0.203962
2021-12-30  000001    0.204713
Name: close, Length: 1215, dtype: float64

In [93]:
%%time
# df_all.loc[(slice('2017-01-10'),['000001','000002','000063']),:]
# pd.concat(indxes,axis=1).sort_index().loc[(slice('2017-01-15'),['000001','000002','000063']),:]

# # a = load_cache('alpha11_train', cache_type=CACHE_TYPE.FACTOR)
# # b = load_cache('alpha24_train', cache_type=CACHE_TYPE.FACTOR)
# pd.concat([a,b],axis=1).sort_index().loc[(slice('2017-01-20'),['000001','000002']),:]
# pd.Series(-1,index=df_treated.loc[(slice(None),['000001']),:].index,dtype=np.dtype('int8'))

# a=alpha004(df_treated.loc[(slice(None),['000001']),:])
# a.rolling(window=5, min_periods=5).corr(a).to_list()
# a.to_list()
# a.rolling(window=5, min_periods=5).apply(lambda x:x.corr(a[x.index])).to_list()
# a[0:50].rolling(window=5, min_periods=5).apply(lambda x:print(x.corr(a[x.index])))


# def alpha083(data, dependencies=['high', 'volume'], max_window=5):
#     # (-1*RANK(COVIANCE(RANK(HIGH),RANK(VOLUME),5)))
#     alpha = COVIANCE(data['high'].rank(pct=True),data['volume'].rank(pct=True),5)*-1
#     return alpha

# print(alpha083(df_treated.loc[(slice(None),['000001']),:]).to_list()[0:100])

# import matplotlib.pyplot as plt
# plt.plot(np.arange(100),alpha083(df_treated.loc[(slice(None),['000001']),:]).to_list()[0:100])
# plt.show()
# plt.plot(np.arange(100),alpha090(df_treated.loc[(slice(None),['000001']),:]).to_list()[0:100])
# plt.show()


# X = df_treated.loc[(slice(None),['000001']),:]['close']
# Y = pd.Series(np.random.rand(1215),index=X.index)
# X = pd.Series(np.random.rand(10000000))
# Y = pd.Series(np.random.rand(10000000))

# x=np.array(range(1, 10))
# w = np.array(range(1, 10))


Wall time: 0 ns


In [94]:
%%time
# b=akshare.stock_zh_a_tick_163(code="sz003023", trade_date="20221025") 
# c = akshare.stock_zh_a_tick_tx_js(code="sz003023")
# c['成交时间']
# b['时间']

# X2=X.to_numpy().reshape(-1, 1)
# Y2=Y.to_numpy().reshape(-1, 1)

# reg = linear_model.LinearRegression(fit_intercept=False)
# reg.fit(X2,Y2).coef_

# a = df_treated.loc[(slice(None),['000001']),:]
# x = a['close']
# y = a['Open']





Wall time: 0 ns


In [89]:
%%time
w = np.array(range(1, 21))
w = w/w.sum()
w

Wall time: 0 ns


array([0.0047619 , 0.00952381, 0.01428571, 0.01904762, 0.02380952,
       0.02857143, 0.03333333, 0.03809524, 0.04285714, 0.04761905,
       0.05238095, 0.05714286, 0.06190476, 0.06666667, 0.07142857,
       0.07619048, 0.08095238, 0.08571429, 0.09047619, 0.0952381 ])

In [90]:
%%time
w = preprocessing.normalize(np.array([i for i in range(1, 21)]).reshape(-1, 1), norm='l1', axis=0).reshape(-1)
w

Wall time: 0 ns


array([0.0047619 , 0.00952381, 0.01428571, 0.01904762, 0.02380952,
       0.02857143, 0.03333333, 0.03809524, 0.04285714, 0.04761905,
       0.05238095, 0.05714286, 0.06190476, 0.06666667, 0.07142857,
       0.07619048, 0.08095238, 0.08571429, 0.09047619, 0.0952381 ])